In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd drive/My Drive/수업/DS_2021/PyTorch

In [ ]:
# Lab 10 MNIST and Deep learning CNN
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import torch.nn as nn

# How to use TensorBoard with PyTorch
TensorBoard is a visualization toolkit for machine learning experimentation. TensorBoard allows tracking and visualizing metrics such as loss and accuracy, visualizing the model graph, viewing histograms, displaying images and much more. In this tutorial we are going to cover TensorBoard installation, basic usage with PyTorch, and how to visualize data you logged in TensorBoard UI.

## Using TensorBoard in PyTorch
Let’s now try using TensorBoard with PyTorch! Before logging anything, we need to create a SummaryWriter instance.



In [ ]:
%matplotlib inline
from tensorflow import summary
%load_ext tensorboard

In [ ]:
train_log_dir = 'logs/tensorboard/train/'
test_log_dir = 'logs/tensorboard/test/' 
train_summary_writer = summary.create_file_writer(train_log_dir)
test_summary_writer = summary.create_file_writer(test_log_dir)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [ ]:
# parameters
learning_rate = 0.001
training_epochs = 20
batch_size = 100

In [ ]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [ ]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [ ]:
# CNN Model
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.5

        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer2 = torch.nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))


        self.layer4 = nn.Sequential(
            nn.Linear(3 * 3 * 128, 625, bias=True),
            nn.ReLU(),
            nn.Dropout(p=1 - self.keep_prob),
            nn.Linear(625, 10, bias=True)
            )

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.layer4(out)
        return out

In [ ]:
# instantiate CNN model
model = CNN().to(device)

In [ ]:
pip install pytorch_model_summary

In [ ]:
import pytorch_model_summary
print(pytorch_model_summary.summary(model, torch.zeros(1, 1, 28, 28).to(device), show_input=False))

In [ ]:
# define cost/loss & optimizer
criterion = nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
%tensorboard --logdir logs/tensorboard

In [ ]:
# train my model
total_batch = len(data_loader)
model.train()    # set the model to train mode (dropout=True)
print('Learning started. It takes sometime.')

for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
    
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost 
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost/total_batch))
    with train_summary_writer.as_default():
        x=avg_cost/total_batch
        summary.scalar('loss', x.cpu().detach().numpy(), step=epoch)
print('Learning Finished!')